In [80]:
from analysis_functions import *
import numpy as np
import CoolProp.CoolProp as CP
from cycle_functions import *
from scipy.optimize import minimize, Bounds, NonlinearConstraint
import warnings
import pandas as pd
import numpy as np
import CoolProp.CoolProp as CP
from cycle_functions import *
from scipy.optimize import minimize, Bounds, NonlinearConstraint, LinearConstraint
import warnings
import pandas as pd


def make_cycle(Vars, Inputs, Param):

    # ----------------------------------------------#
    # ==------ Vars  -------==#
    P_c    = Vars[0] # Pa
    P_e    = Vars[1] # Pa
    T_SH   = Vars[2] # delta-T K
    # ----------------------------------------------#
    #==------ Inputs ------==#
    
    Q_load = Inputs[0] # W
    T_amb  = Inputs[1] # K
    T_pod  = Inputs[2] # K
    U_cond = Inputs[3] # m/s
    
    #----------------------------------------------#
    #==------ Param -------==#
    RPM    = Param[0]
    
    
    #----------------------------------------------#
    #==-- Init. Outputs  --==#
    P = np.zeros(9) # Pa
    T = np.zeros(9) # K
    h = np.zeros(9) # j/kg
    s = np.zeros(9) # j/kg/k
    abscissa = np.zeros(9)
    # var "abscissa" is the nondimensional 
    # Heat exchanger position 
    # for each of these stations
    # domain = [0,1]U[1,2]
    # [0,1] <-- in condensor
    # [1,2] <-- in evaporator
    
    #=========================================================================#
    # Calculate
    #=========================================================================#

    # pressure drop accross evaporator (Pa)
    delta_P_e = 5e4
    
    # pressure drop accross condenser (Pa)
    delta_P_c = 5e4
    
    P[0] = P_e - delta_P_e # Pressure drop accross evap determined empirically
    
    
    # Init state
    T_sat_e = CP.PropsSI('T', 'P', P[0], 'Q', 1, 'R410a') # K
    h_g     = CP.PropsSI('H', 'P', P[0], 'Q', 1, 'R410a') # J/kg
    T[0] = T_sat_e + T_SH
    h[0] = CP.PropsSI('H', 'P', P[0], 'T', T[0], 'R410a')
    abscissa[0] = 0
    s[0] = CP.PropsSI('S', 'P', P[0], 'H', h[0], 'R410a')
    
    STATE   = [P[0], h[0]]
    
    #   calculate compressor
    m_dot_s = compr_func(STATE, RPM, P_c / P[0])
    P[1] = P_c
    
    # Isentropic Ratio
    eta_is = 2.9
    
    if eta_is < 1:
        print([RPM, P_c, P_e])
   
    h[1] = h[0] + (CP.PropsSI('H', 'P', P_c, 'S', s[0], 'R410a') - h[0]) / eta_is
    s[1] = CP.PropsSI('S', 'P', P[1], 'H', h[1], 'R410a')

    STATE = [P[1], h[1]]
    
    
    #   calculate condenser
    [P[1:5], T[1:5], h[1:5], s[1:5], abscissa[1:5]] = Condenser_Proc( STATE, 
                                                             'h', m_dot_s, T_amb, U_cond)


    #   calculate expansion
#     m_dot_v = valve_func( CA, P_c, P_e, valve )
    m_dot_v = capillary_tube_func(P[4], h[4], T[4])
    
    P[5] = P_e
    # Isenthalpic expansion
    h[5] =  h[4]
    
    STATE = [P[5], h[5]]
    

    #   calculate evap
    [P[5:9], T[5:9], h[5:9], s[5:9], abscissa[5:9]] = Evap_Proc(STATE, m_dot_s, T_pod)

    abscissa[5:9] = abscissa[5:9] + abscissa[4]

    # Energy and Mass Deficits
    Q_evap = m_dot_s * (h[8] - h[5])
    Q_absr = m_dot_s * (h[0] - h[5])

    m_def  =  (m_dot_s - m_dot_v) / m_dot_s  #Mass Deficit
    h_def  =  (Q_absr  - Q_evap) / Q_evap   #evap deficit
    Q_def  =  (Q_evap  - Q_load) / Q_load   #Pod energy deficit

    Deficit = np.array([m_def, h_def, Q_def])

    #Other Outputs
    m_dot = [m_dot_s, m_dot_v]
    Q_L   = Q_evap
    Q_H   = m_dot_v * (h[1] - h[4])
    
    # Compute compressor work based on isentropic, adiabatic compressor
    W     = m_dot_s * (CP.PropsSI('H', 'P', P_c, 'S', s[0], 'R410a') - h[0])



    return [P, T, h, s, abscissa, m_dot, Q_L, Q_H, W, Deficit]


def Condenser_Proc(input_state, strarg, flowrate, T_amb, airspeed):

    # Input state must be a row vector containing pressure 
    # and enthalpy in that order
    # input_state = [P, h]
    
    #Initialize Vars
    #----------------------
    P_in = input_state[0]
    P = P_in * np.ones(4)
    h = np.zeros(4)
    T = np.zeros(4)
    s = np.zeros(4)

    abcissa = np.zeros(4)
    dz_1 = 0
    dz_2 = 0
    dz_3 = 0

    #=========================================================================#
    # set up us the properties

    if strarg == 'h':

        h_in = input_state[1]
        T_in = CP.PropsSI('T', 'P', P_in, 'H', h_in, 'R410a')
        T_sat = CP.PropsSI('T', 'P', P_in, 'Q', 1, 'R410a')
        h_f   = CP.PropsSI('H', 'P', P_in, 'Q', 0, 'R410a')
        h_g   = CP.PropsSI('H', 'P', P_in, 'Q', 1, 'R410a')
        h_fg  = h_g - h_f    
 
    
#         T_in  = fsolve(lambda t: ((h_in-h_g) - SuperHT_Cp_integral(T_sat, t)), 
#                        T_sat + 1)


        # assign output
        #----------------
        T[0] = T_in;
        h[0] = h_in;
        #----------------

    elif strarg == 'T':

        T_in = input_state[1];
        h_in = CP.PropsSI('H', 'P', P_in, 'T', T_in, 'R410a')
        T_sat = CP.PropsSI('T', 'P', P_in, 'Q', 1, 'R410a')
        h_f   = CP.PropsSI('H', 'P', P_in, 'Q', 0, 'R410a')
        h_g   = CP.PropsSI('H', 'P', P_in, 'Q', 1, 'R410a')
        h_fg  = h_g - h_f;

#         h_in  = h_g + SuperHT_Cp_integral(T_sat, T_in);


        # assign output
        #----------------
        T[0] = T_in;
        h[0] = h_in;
        #----------------

    else:
        raise ValueError('dont recognize input property' + strarg)



    #=========================================================================#
    # Calculate Vars
    #

    [UA_1, UA_3] = generate_HTCOEFF( P_in, flowrate, flowrate, 
                                       airspeed, 'COND', T_amb)

    #Temporary
    UA_g = UA_1
    UA_f = UA_3

    #Properties
    c_p_g = 0.5 * (CP.PropsSI('C', 'P', P_in, 'H', h_in, 'R410a') + 
                   CP.PropsSI('C', 'P', P_in, 'Q', 1, 'R410a'))
    c_p_f = CP.PropsSI('C', 'P', P_in, 'Q', 0, 'R410a')

    rho_g   = CP.PropsSI('D', 'P', P_in, 'Q', 1, 'R410a')
    rho_f   = CP.PropsSI('D', 'P', P_in, 'Q', 0, 'R410a')
    #rho_fg  = rho_f - rho_g; or rho_g - rho_f?
    rho_rat = rho_g/rho_f

    #Vol Void Frac
    gamma = 1 / (1 - rho_rat) + rho_rat / (rho_rat - 1)**2 * np.log( rho_rat )

    UA_2 = UA_f * (1 - gamma) + UA_g * (gamma)




    #=========================================================================#
    #
    #  begin integration procedure, piecewise
    #
    #

    #--- Superheat-into-Saturation Process ---
    # Check that ambiet temperature is above the saturation 
    # and inlet temperature otherwise go straight to subcooled
    if (T_amb - T_in) < 0  and (T_amb - T_sat) < 0:
        dz_1 = c_p_g  * flowrate / UA_1 * np.log((T_amb - T_in) / 
                                                 (T_amb - T_sat))

        #Add exception if superheated phase takes up the
        #entire HX domain
        if (dz_1 > 1):
            T = np.nan
            h = np.nan
            P = np.nan
            abcissa = np.nan
            raise ValueError('no exception when superheated' +
                             ' phase takes up entire domain')


        T[1] = T_sat
        h[1] = h_g


        #--- SatVap-into-SatLiq Process ---

        dz_2 = flowrate * h_fg / (UA_2 * (T_sat - T_amb))

            #Begin exception if saturation phase takes up the 
            #remainder of the HX domain
        if (dz_1 + dz_2) > 1:

            dz_2   = 1 - dz_1

            #solve system 
            #gamma and delta_h are the variables\
            x = lambda dh: (dh + h_fg) / h_fg # x(var[1])
            f = lambda var: [dz_2 * (T_amb - T_sat) * 
                             (UA_f + (UA_g - UA_f) * var[0]) - 
                             (flowrate * var[1]),

                             (1 - x(var[1])) * 
                             (1 / (1 - rho_rat) - var[0]) + 
                             rho_rat / (rho_rat - 1)**2 * 
                             np.log(rho_rat + (1 - rho_rat) * 
                                    x(var[1]))
                            ]

            b = fsolve( f, [gamma, -h_fg] )
            # gamma = b(1);
            dh_2  = b[1]

            #-----------------
            # Produce Output
            #
            h_out = h_g + dh_2;
            #
            # assign output
            #-----------------
            T[2] = T_sat
            h[2] = h_out
            T[3] = T[2]
            h[3] = h[2]
            #-----------------

            #Otherwise go to subcool process  
        else:

            # assign output
            #-----------------
            T[2] = T_sat
            h[2] = h_f
            #-----------------      



    #--- SatLiq-into-Subcool Process ---        

    dz_3 = 1 - dz_1 - dz_2

    T_out = (T_sat - T_amb) * np.exp(-UA_3 / (c_p_f * 
                                              flowrate) * dz_3) + T_amb
    h_out = h_f + c_p_f * (T_out - T_sat)


    # assign output
    #-----------------
    T[3] = T_out;
    h[3] = h_out;
    
    # Pressure drop determined empirically applied linearly
    P[1] = P[0] - 5e4 * dz_1
    P[2] = P[1] - 5e4 * dz_2
    P[3] = P[2] - 5e4 * (1 - dz_2 + dz_1)
    #-----------------


    # assign output
    #-----------------------------------
    abcissa[1] = abcissa[0] + dz_1
    abcissa[2] = abcissa[1] + dz_2
    abcissa[3] = 1
    #-----------------------------------    

    s = CP.PropsSI('S', 'P', P, 'H', h, 'R410a')


    return [P, T, h, s, abcissa]


def generate_HTCOEFF(P, m_dot_g, m_dot_f, V_extr, subsys, T_air):


    if subsys == 'EVAP':
        
        # Geometric Characteristics
        
        # Fin density (fins/m) [measured 15 fins per inch]
        Nf = 20 / 0.0254

        # Outside diameter of tubing (m) [measured .31"]
        do = 0.31 *.0254

        # Inside diameter of tubing (m) [wall thickness estimated at 0.03"]
        di = do - 2 * 0.03 *.0254

        # Transverse spacing between tubes (m) [measured 0.86"]
        xt = 0.86 * 0.0254

        # Longitudinal spacing between tubes (m) [(measured 0.994") / 2]
        xl = (0.994 * 0.0254) / 2

        # Fin thickness (m) [measured 0.004"]
        delta = 0.004 * 0.0254

        # Overall Length (m) 
        L1 = (12.5) * 0.0254
        
        # Overall depth (m) [measured 1 1/2"]
        L2 = (1.5) * 0.0254

        # Overall height (m) 
        L3 = 8.5 * 0.0254
        
        # Number of Rows 
        Nr = 3
        
        # Number of tubes
        Nt = 30

        #Interior (refrigerant side)
        A_i = np.pi * di * Nt * L1 # [m2]

        #Pipe Wall
        k_pipe = 385 # copper [W/m-K]

        R_tw = np.log(do / di) / (2 * np.pi * k_pipe * Nt * L1) # [K/W]

        #Exterior (air side)
        
        # Primary surface area (tubes and header plates)
        A_p = np.pi * do * (L1 - delta * Nf * L1) * Nt + 2 * (L3 * L2 - np.pi * do**2 / 4 * Nt)

        # Secondary surface area (fins)
        A_f = 2 * (L2 * L3 - (np.pi * do**2 /  4) * Nt) * Nf * L1 + 2 * L3 * delta * Nf * L1 
        
        A_a = A_f + A_p #[m2] #Heat transfer area airside
        
        # Volume occupied by the heat exchanger (heat exchanger total volume) (m^3)
        V_a = L1 * L2 * L3
        
        ## Outside geometric characteristics

        # Minimum free-flow area (Fundamentals of Heat Exchanger Design-Shah pg 573) 

        # 2a''
        a_prime = (xt - do) - (xt - do) * delta * Nf 

        # 2b''
        b_prime = 2 * (((xt / 2) ** 2 + xl ** 2) ** 0.5 - do - (xt - do) * delta * Nf)

        # c''
        if a_prime < b_prime:
            c_prime = a_prime
        else:
            c_prime = b_prime

        # Minimum free-flow area (m^2)
        A_o_a = ((L3 / xt - 1) * c_prime + (xt - do) - (xt - do) * delta * Nf) * L1

        # Frontal area (m^2)
        A_fr_a = L1 * L3

        # Ratio of free flow area to frontal area
        sigma_a  = A_o_a / A_fr_a

        # surface area density 
        alpha_a = A_a / V_a

        # Hydralic diameter (m)
        D_h_a = 4 * sigma_a / alpha_a

        #Need Adjusted airspeed based on obstructed area

        adjspeed = V_extr / sigma_a

        #-------------------------------------------------------------------------#
        # Refrigerant Constants (R410a) 
        #-------------------------------------------------------------------------#

        k_f  = CP.PropsSI('L', 'P', P, 'Q', 0, 'R410a') # [W/m-K] 
        k_g  = CP.PropsSI('L', 'P', P, 'Q', 1, 'R410a') # [W/m-K] 
        mu_f = CP.PropsSI('V', 'P', P, 'Q', 0, 'R410a') # [Pa-s] 
        mu_g = CP.PropsSI('V', 'P', P, 'Q', 1, 'R410a') # [Pa-s]
        c_p_f = CP.PropsSI('C', 'P', P, 'Q', 0, 'R410a') #[J/kg-K]  
        c_p_g = CP.PropsSI('C', 'P', P, 'Q', 1, 'R410a') #[J/kg-K]           

        #-------------------------------------------------------------------------#
        # Air Constants
        #-------------------------------------------------------------------------#

        k_a = CP.PropsSI('L', 'P', 101325, 'T', T_air, 'air') #[W/m-K]   
        mu_a = CP.PropsSI('V', 'P', 101325, 'T', T_air, 'air') #[Pa-s]   
        rho_a = CP.PropsSI('D', 'P', 101325, 'T', T_air, 'air') #[kg/m3] 
        c_p_a = CP.PropsSI('C', 'P', 101325, 'T', T_air, 'air') #[J/kg-K]
        Pr_a = CP.PropsSI('Prandtl', 'P', 101325, 'T', T_air, 'air') #[J/kg-K]

        #-------------------------------------------------------------------------#
        # Derived Relations
        #-------------------------------------------------------------------------#
        
        # fluid mass velocity (kg/(m^2 s))
        G_a = A_fr_a * V_extr * rho_a / A_o_a

        # Compute Reynold's number
        Re_a = G_a * D_h_a / mu_a

        # Compute j using equation 7.141 Nr >= 2 (Fundamentals of Heat Exchanger Design-Shah pg 551)

        # collar diameter (m)
        dc = do + 2 * delta

        # fluid mean axial velocity (m/s)
        u_m = adjspeed

        # Collar Reynolds number
        Re_dc = rho_a * u_m * dc / mu_a

        # fin pitch (m/fin)
        pf = 1 / Nf

        # constants from equation
        C3 = -0.361 - 0.042 * Nr / np.log(Re_dc) + 0.158 * np.log(Nr * (pf / dc) **0.41)

        C4 = -1.224 - (0.076 * (xl / D_h_a) ** 1.42) / np.log(Re_dc)

        C5 = -0.083 + 0.058 * Nr / np.log(Re_dc)

        C6 = -5.735 + 1.21 * np.log(Re_dc / Nr)

        # Compute outside heat transfer coefficeinet using colburn j factor (more accurate)
        j = 0.086 * Re_dc ** C3 * Nr ** C4 * (pf / dc) ** C5 * (pf / D_h_a) ** C6 * (pf / xt) ** -0.93

        # h = JGCp/Pr^2/3
        h_a = j * G_a * c_p_a / Pr_a**(2/3)
        
#         Nu_a  =  Circular_Duct_Nu([Re_a], [Pr_a], 'c')
        
#         h_a = Nu_a * k_a / D_h_a

        # Single fin efficiency 
        # (Fundamentals of Heat Exchanger Design-Shah pg 606 eqn 9.14)
        m = (2 * h_a / k_pipe / delta) ** 0.5

        l = xl / 2 - delta 

        # Determine single fin efficiency
        eta_f = np.tanh(m * l) / (m * l)
        
        #Overall Fin efficiency
        fin_eff = 1 - (1 - eta_f) * A_f / A_a

#         print(j)
#         print(h_a)
#         print(A_a)
#         print(fin_eff)
        
        addcnst = A_i * R_tw + A_i / (h_a * fin_eff * A_a)

        #HT-coefficient, gaseous, contribution from refrigerant side
        Re_g  =  4 * m_dot_g / (np.pi * di * mu_g)
        Pr_g  =  c_p_g * mu_g / k_g
        Nu_g  =  Circular_Duct_Nu([Re_g], [Pr_g], 'h')  
        h_i_g =  k_g * Nu_g / di


        #HT-coefficient, liquid, contribution from refrigerant side
        Re_f  =  4 * m_dot_f / (np.pi * di * mu_f)
        Pr_f  =  c_p_f * mu_f / k_f
        Nu_f  =  Circular_Duct_Nu([Re_f], [Pr_f], 'h')  
        h_i_f =  k_f * Nu_f / di


        #Local overall heat transfer coefficient
        U_g = (1 / h_i_g + addcnst)**-1
        U_f = (1 / h_i_f + addcnst)**-1

        #Output UA
        UA_g = U_g*A_i
        UA_f = U_f*A_i

    elif subsys == 'COND':

        # Geometric Characteristics
        
        # Fin density (fins/m) [measured 20 fins per inch]
        Nf = 19 / 0.0254

        # Outside diameter of tubing (m) [measured .21"]
        do = 0.21 * 0.0254

        # Inside diameter of tubing (m) [wall thickness estimated at 0.03"]
        di = do - 2 * 0.03 *.0254

        # Transverse spacing between tubes (m) [measured 1.048" - do]
        xt = 1.048 * 0.0254 - do

        # Longitudinal spacing between tubes (m) [(measured 1.066" - do) / 2]
        xl = (1.066 * 0.0254 - do) / 2

        # Fin thickness (m) [measured 0.004"]
        delta = 0.004 * 0.0254

        # Overall Length (m) [measured 15 15/16"]
        L1 = (15 + 15/16) * 0.0254
        
        # Overall depth (m) [measured 1 5/16"]
        L2 = (1 + 5/16) * 0.0254

        # Overall height (m) [measured 12.5"]
        L3 = 12.5 * 0.0254
        
        # Number of Rows 
        Nr = 3
        
        # Number of tubes
        Nt = 44 - 2


        #Interior (refrigerant side)
        A_i     = np.pi * di * Nt * L1 # [m2]

        #Pipe Wall
        k_pipe = 385 # copper [W/m-K]

        R_tw = np.log(do / di) / (2 * np.pi * k_pipe * Nt * L1) # [K/W]

        #Exterior (air side)
        
        # Primary surface area (tubes and header plates)
        A_p = np.pi * do * (L1 - delta * Nf * L1) * Nt + 2 * (L3 * L2 - np.pi * do**2 / 4 * Nt)

        # Secondary surface area (fins)
        A_f = 2 * (L2 * L3 - (np.pi * do**2 / 4) * Nt) * Nf * L1 + 2 * L3 * delta * Nf * L1 
        
        A_a = A_f + A_p #[m2] #Heat transfer area airside
        
        # Volume occupied by the heat exchanger (heat exchanger total volume) (m^3)
        V_a = L1 * L2 * L3
        
        ## Outside geometric characteristics

        # Minimum free-flow area (Fundamentals of Heat Exchanger Design-Shah pg 573) 

        # 2a''
        a_prime = (xt - do) - (xt - do) * delta * Nf 

        # 2b''
        b_prime = 2 * (((xt / 2) ** 2 + xl ** 2) ** 0.5 - do - (xt - do) * delta * Nf)

        # c''
        if a_prime < b_prime:
            c_prime = a_prime
        else:
            c_prime = b_prime

        # Minimum free-flow area (m^2)
        A_o_a = ((L3 / xt - 1) * c_prime + (xt - do) - (xt - do) * delta * Nf) * L1

        # Frontal area (m^2)
        A_fr_a = L1 * L3

        # Ratio of free flow area to frontal area
        sigma_a  = A_o_a / A_fr_a

        # surface area density 
        alpha_a = A_a / V_a

        # Hydralic diameter (m)
        D_h_a = 4 * sigma_a / alpha_a

        #Need Adjusted airspeed based on obstructed area

        adjspeed = V_extr / sigma_a

        #-------------------------------------------------------------------------#
        # Refrigerant Constants (R410a) 
        #-------------------------------------------------------------------------#

        k_f  = CP.PropsSI('L', 'P', P, 'Q', 0, 'R410a') # [W/m-K] 
        k_g  = CP.PropsSI('L', 'P', P, 'Q', 1, 'R410a') # [W/m-K] 
        mu_f = CP.PropsSI('V', 'P', P, 'Q', 0, 'R410a') # [Pa-s] 
        mu_g = CP.PropsSI('V', 'P', P, 'Q', 1, 'R410a') # [Pa-s]
        c_p_f = CP.PropsSI('C', 'P', P, 'Q', 0, 'R410a') #[J/kg-K]  
        c_p_g = CP.PropsSI('C', 'P', P, 'Q', 1, 'R410a') #[J/kg-K]           

        #-------------------------------------------------------------------------#
        # Air Constants
        #-------------------------------------------------------------------------#

        k_a = CP.PropsSI('L', 'P', 101325, 'T', T_air, 'air') #[W/m-K]   
        mu_a = CP.PropsSI('V', 'P', 101325, 'T', T_air, 'air') #[Pa-s]   
        rho_a = CP.PropsSI('D', 'P', 101325, 'T', T_air, 'air') #[kg/m3] 
        c_p_a = CP.PropsSI('C', 'P', 101325, 'T', T_air, 'air') #[J/kg-K]
        Pr_a = CP.PropsSI('Prandtl', 'P', 101325, 'T', T_air, 'air') #[J/kg-K]

        #-------------------------------------------------------------------------#
        # Derived Relations
        #-------------------------------------------------------------------------#
        
        # fluid mass velocity (kg/(m^2 s))
        G_a = A_fr_a * V_extr * rho_a / A_o_a

        # Compute Reynold's number
        Re_a = G_a * D_h_a / mu_a

        # Compute j using equation 7.141 Nr >= 2 (Fundamentals of Heat Exchanger Design-Shah pg 551)

        # collar diameter (m)
        dc = do + 2 * delta

        # fluid mean axial velocity (m/s)
        u_m = adjspeed

        # Collar Reynolds number
        Re_dc = rho_a * u_m * dc / mu_a

        # fin pitch (m/fin)
        pf = 1 / Nf

        # constants from equation
        C3 = -0.361 - 0.042 * Nr / np.log(Re_dc) + 0.158 * np.log(Nr * (pf / dc) **0.41)

        C4 = -1.224 - 0.076 * (xl / D_h_a) ** 1.42 / np.log(Re_dc)

        C5 = -0.083 + 0.058 * Nr / np.log(Re_dc)

        C6 = -5.735 + 1.21 * np.log(Re_dc / Nr)

        # Compute outside heat transfer coefficeinet using coburn j factor (more accurate)
        j = 0.086 * Re_dc ** C3 * Nr ** C4 * (pf / dc) ** C5 * (pf / D_h_a) ** C6 * (pf / xt) ** -0.93

        # h = JGCp/Pr^2/3
        h_a = j * G_a * c_p_a / Pr_a ** (2/3)
        
#         Nu_a  =  Circular_Duct_Nu([Re_a], [Pr_a], 'h')
        
#         h_a = Nu_a * k_a / D_h_a

        # Single fin efficiency 
        # (Fundamentals of Heat Exchanger Design-Shah pg 606 eqn 9.14)
        m = (2 * h_a / k_pipe / delta) ** 0.5

        l = xl / 2 - delta 

        # Determine single fin efficiency
        eta_f = np.tanh(m * l) / (m * l)
        
        #Overall Fin efficiency
        fin_eff = 1 - (1 - eta_f) * A_f / A_a
        
#         print(j)
#         print(h_a)
#         print(A_a)
#         print(fin_eff)
        
        addcnst = A_i * R_tw + A_i / (h_a * fin_eff * A_a)

        #HT-coefficient, gaseous, contribution from refrigerant side
        Re_g  =  4 * m_dot_g / (np.pi * di * mu_g)
        Pr_g  =  c_p_g * mu_g / k_g
        Nu_g  =  Circular_Duct_Nu([Re_g], [Pr_g], 'c')  
        h_i_g =  k_g * Nu_g / di


        #HT-coefficient, liquid, contribution from refrigerant side
        Re_f  =  4 * m_dot_f / (np.pi * di * mu_f)
        Pr_f  =  c_p_f * mu_f / k_f
        Nu_f  =  Circular_Duct_Nu([Re_f], [Pr_f], 'c')  
        h_i_f =  k_f * Nu_f / di


        #Local overall heat transfer coefficient
        U_g = ( 1 / h_i_g + addcnst )**-1;
        U_f = ( 1 / h_i_f + addcnst )**-1;


        #Output UA
        UA_g = U_g * A_i
        UA_f = U_f * A_i

    else:
        raise ValueError('Subsys must be "COND" or "EVAP"')


    return [UA_g, UA_f]

In [81]:
path = 'C:/Users/charl/Google Drive/school/Graduate/Pod Project/Prototype/'

# list
files =['11-10-2020/test.lvm', '11-11-2020/test.lvm', '11-12-2020/test.lvm', '11-16-2020/test.lvm',
        '11-16-2020/test2.lvm', '11-17-2020/test.lvm', '11-17-2020/test2.lvm', '11-18-2020/test.lvm',
        '11-18-2020/test2.lvm', '11-18-2020/test3.lvm']

P_ambs = [101.35e3, 100.83e3, 101.12e3, 100.41e3,
          100.51e3, 100.67e3, 100.69e3, 102.37e3,
          102.40e3, 102.46e3, ]

Input_Qs = [1366, 1365, 1386, 1396,
            1394, 1385, 1392, 1415,
            1423, 1450]

experimentalData = pd.read_pickle(path + 'experimentalDataframe.pkl')

for ind, file in enumerate(files):
    
    if file not in experimentalData['file'].values:
        experimentalData = experimentalData.append(experimental_analysis(path + file, P_ambs[ind], Input_Qs[ind]))
        
        
experimentalData.to_pickle(path + 'experimentalDataframe.pkl')

experimentalData = experimentalData.reset_index()

In [82]:
modelData = pd.DataFrame()

for index, row in experimentalData.iterrows():
    
    if index == 1:

        Inputs = np.array([row['Total Heat Load (W)'],
                           row['Ambient T (K)'],
                           row['Pod T (K)'],
                           row['Wind Tunnel Velocity (m/s)'],
                          ])

        Param = np.array([3550])

        [ P, T, h, s, abcissa, m_dot, Q_L, Q_H, W, Deficit] = make_cycle([row['P (Pa)'][1], row['P (Pa)'][0] + 5e4, 19],
                                                                             Inputs,
                                                                             Param)

        modelData = modelData.append(pd.DataFrame({'P (Pa)': [P], 'T (K)': [T], 'h (j/kg)': [h], 's (j/kg K)': [s], 'abcissa': [abcissa], 
                                      'mass flux (kg/s)': [m_dot], 'Evaporator Heat transfer (W)': Q_L, 'Condenser Heat transfer (W)': Q_H, 'Compressor Work (W)': W, 
                                      'Deficits': [Deficit]}))

        print(Deficit)
        print(experimentalData.iloc[index]['Wind Tunnel Velocity (m/s)'])
        print(m_dot)
        print(Q_L)
        print(W)



        thermodynamic_plots(experimentalData.iloc[index], modelData.iloc[-1], lgnd = ['Vapor Dome', 'Refrigeration Cycle Model',
                            'Refrigeration Cycle Measured', 'Ambient Temperature',
                           'Pod Temperature'], annotate = True, color ='r')


        modelData = modelData.reset_index()

[ 0.00055959  0.01456437 -0.02326751]
5.362611509198611
[0.004518632703487711, 0.00451610411591436]
954.8393146842404
129.61478222963905


In [6]:
path = 'C:/Users/charl/Google Drive/school/Graduate/Pod Project/Prototype/'
experimentalData = pd.read_pickle(path + 'experimentalDataframe.pkl')
experimentalData = experimentalData.reset_index()

modelData = pd.DataFrame()

for index, row in experimentalData.iterrows():

    if index == 7:
        
        Inputs = np.array([row['Total Heat Load (W)'],
                           row['Ambient T (K)'],
                           row['Pod T (K)'],
                           row['Wind Tunnel Velocity (m/s)'],
                          ])

        Param = np.array([3550])


        T_amb  = Inputs[1] # K
        T_pod  = Inputs[2] # K

        SPREAD = 4;

        #Var Extents
        lb = [CP.PropsSI('P', 'T', T_amb, 'Q', 1, 'R410a'), 400e3]
        ub = [5000e3, CP.PropsSI('P', 'T', T_pod, 'Q', 0, 'R410a')]

        #Starting points
        P_c   = lb[0] + (ub[0] - lb[0]) * np.linspace( 0.1, 0.9, SPREAD)
        P_e   = lb[1] + (ub[1] - lb[1]) * np.linspace( 0.1, 0.9, SPREAD)
        T_SH  = .5

        # Create list of possible combinations of pressures
        Vars = np.array(np.meshgrid(P_c, P_e, T_SH)).T.reshape(-1, 3)

        #Initialize Vars and Deficits
        normDeficit = np.zeros(len(Vars))
        Deficit     = np.zeros((len(Vars), 3))

        # Try different initial points
        for ind, Var in enumerate(Vars):
            #Step Vars Forward
            [Vars[ind], Deficit[ind]] = adjust_cycle_fmin( Var, Inputs, Param)
            normDeficit[ind] = np.linalg.norm(Deficit[ind])


        # find solution with lowest error
        Vars = Vars[normDeficit == np.nanmin(normDeficit)][0]

        # Check if error is lower than 5% 
        converged = 1
        if normDeficit[normDeficit == np.nanmin(normDeficit)] > 0.05:
            converged = 0
            warnings.warn('Warning: |Deficit| = ' + 
                          str(normDeficit[normDeficit == min(normDeficit)]))

        #Calc
        [ P, T, h, s, abcissa, m_dot, Q_L, Q_H, W, Deficit] = make_cycle(Vars, 
                                                                         Inputs,
                                                                         Param)
        Props = [P, T, h, s, abcissa]

.\cycle_functions.py:935: UserWarning: Partial Evaporation
  warnings.warn('Partial Evaporation')
C:\ProgramData\Anaconda3\envs\jupyterlab-debugger\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
C:\ProgramData\Anaconda3\envs\jupyterlab-debugger\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


Compression ratio too high
initial Point: [2.31955829e+06 4.71188979e+05 5.00000000e-01]


.\cycle_functions.py:805: UserWarning: Warning cavitation in expansion valve
  warnings.warn('Warning cavitation in expansion valve')


Compression ratio too high
initial Point: [3.44816533e+06 4.71188979e+05 5.00000000e-01]
Compression ratio too high
initial Point: [3.44816533e+06 6.61026255e+05 5.00000000e-01]
Compression ratio too high
initial Point: [4.57677236e+06 4.71188979e+05 5.00000000e-01]
Compression ratio too high
initial Point: [4.57677236e+06 6.61026255e+05 5.00000000e-01]
Compression ratio too high
initial Point: [4.57677236e+06 8.50863531e+05 5.00000000e-01]
Compression ratio too high
initial Point: [4.57677236e+06 1.04070081e+06 5.00000000e-01]


In [2]:
Var = np.array([2.31955829e+06, 6.61026255e+05, 5.00000000e-01])
Inputs = np.array([812.45306455, 271.93939974, 283.87752544,   5.05658304])
Param = np.array([3550])
[Varst, Deficitt] = adjust_cycle_fmin( Var, Inputs, Param)

.\cycle_functions.py:935: UserWarning: Partial Evaporation
  warnings.warn('Partial Evaporation')
C:\ProgramData\Anaconda3\envs\jupyterlab-debugger\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
.\cycle_functions.py:805: UserWarning: Warning cavitation in expansion valve
  warnings.warn('Warning cavitation in expansion valve')
C:\ProgramData\Anaconda3\envs\jupyterlab-debugger\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_gr